### John Hopkins GitHub Repo Data Wrangling

In [1]:
import json
import os
import sys
import pandas as pd
import numpy as np
import boto3
import uuid
from slugify import slugify

In [2]:
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__name__))), 'COVID-19')

In [3]:
# the confirmed cases time series files in here have Lat and Long for each location
# confirmed_series_file = os.path.join(
#     DATA_DIR,
#     'csse_covid_19_data',
#     'csse_covid_19_time_series',
#     'time_series_covid19_confirmed_global.csv'
# )
# confirmed_series_file

In [4]:
# show first 10 rows of time_series_19-covid-Confirmed.csv file
#!awk -F, '{print $1,$2,$3,$4} NR==10{exit}' OFS=', ' \
# COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv

In [5]:
# cols=['Province/State', 'Country/Region', 'Lat', 'Long']
# locations_df = pd.read_csv(confirmed_series_file, usecols=cols)
# locations_df.head()

In [6]:
s3_resource = boto3.resource('s3')
bucket_name = 'thecodinginterface-covid'
s3_bucket = s3_resource.Bucket(name=bucket_name)

In [7]:
def slugify_location(country_region, province_state):
    if province_state:
        return slugify(f"{country_region}-{province_state}")
    return slugify(country_region)

def cloud_resource_url(filename, bucket_name):
    return f"https://{bucket_name}.s3.amazonaws.com/{filename}.json"

def upload_file_to_s3(s3_bucket, file_path, file_name):
    s3_bucket.upload_file(
        Filename=file_path,
        Key=file_name,
        ExtraArgs={'ACL':'public-read'}
    )
    return cloud_resource_url(file_name, s3_bucket.name)

In [8]:
# rename columns to be snake_cased making it more ammenable to serialization
# locations_df = locations_df.rename(columns={
#     'Province/State': 'province_state',
#     'Country/Region': 'country_region',
#     'Lat': 'lat',
#     'Long': 'long'
# })

# # make sure text columns are well cleaned and stripped of whitespace
# locations_df.province_state = locations_df.province_state.str.strip()
# locations_df.country_region = locations_df.country_region.str.strip()

# # Fill NaNs with empty strings in the Province/State columns because this data will
# # be serialized into JSON which does not support NaN
# locations_df.province_state = locations_df.province_state.fillna('')

# # create columns "filename" and "cloud_resource"
# lookup_keys = zip(locations_df.country_region, locations_df.province_state)
# locations_df['location_id'] = [slugify_location(country_region, province_state)
#                             for country_region, province_state in lookup_keys]

# locations_df['cloud_resource'] = [cloud_resource_url(filename, bucket_name)
#                                   for filename in locations_df['location_id'].values]

# locations_df.head()

In [9]:
# locations_df.country_region.unique()

In [10]:
# locations_df = locations_df.set_index('location_id')
# locations_df[locations_df.country_region == 'US'].sort_values('province_state')

In [11]:
# I'll do more with this locations_df DataFrame later after
# constructing country specific case data sets

In [12]:
# build list of daily csv files
daily_series_dir = os.path.join(
    DATA_DIR,
    'csse_covid_19_data',
    'csse_covid_19_daily_reports'
)
daily_csv_files = [file_name
                   for file_name in os.listdir(daily_series_dir) 
                   if file_name.endswith('csv')]
daily_csv_files[:5]

['02-26-2020.csv',
 '02-27-2020.csv',
 '02-18-2020.csv',
 '02-19-2020.csv',
 '03-24-2020.csv']

In [13]:
# take a peek at the structure of a file that will be worked with
os.path.join(daily_series_dir, daily_csv_files[-1])

'/Users/adammcquistan/Code/python/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-08-2020.csv'

In [14]:
#!head ./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/02-26-2020.csv

In [15]:
def calc_differential(x):
    x0 = np.array([0] + x[:-1].tolist())
    dx = x.values - x0
    return dx

In [16]:
# read the daily files into DataFrame objects then concatenate them together
daily_dfs = []
colunns_of_interest = [
    'province_state',
    'country_region',
    'total_confirmed',
    'total_deaths',
    'total_recovered',
    'date'
]

for file_name in daily_csv_files:
    file_path = os.path.join(daily_series_dir, file_name)
    day_df = pd.read_csv(file_path)

    # Province_State and Country_Region replaced column names Province/State
    # and Country/Region for new daily files starting 03-24-2020
    day_df = day_df.rename(columns={
            'Province/State': 'province_state',
            'Province_State': 'province_state', 
            'Country/Region': 'country_region',
            'Country_Region': 'country_region',
            'Confirmed': 'total_confirmed',
            'Deaths':'total_deaths',
            'Recovered': 'total_recovered'
        })
    
    date_str, ext = os.path.splitext(file_name)
    num_rows = day_df.shape[0]
    day_df['date'] = [pd.to_datetime(date_str)] * num_rows

    # Fill NaNs with empty strings because this data will
    # be serialized into JSON which does not support NaN
    day_df.province_state = day_df.province_state.fillna('')
    day_df.total_confirmed = day_df.total_confirmed.fillna(0)
    day_df.total_deaths = day_df.total_deaths.fillna(0)
    day_df.total_recovered = day_df.total_recovered.fillna(0)

    missing_columns = sum([(col not in day_df.columns) for col in colunns_of_interest])
    if missing_columns:
        import pdb; pdb.set_trace()
        sys.exit(0)

    whole_country_df = day_df[colunns_of_interest].groupby(['country_region']).sum()
    whole_country_df = whole_country_df.reset_index()
    whole_country_df['province_state'] = [''] * whole_country_df.shape[0]
        
    if 'Admin2' in day_df.columns:
        county_df = day_df[day_df.Admin2 != '']
        county_df['province_state'] = county_df.Admin2 + ', ' + county_df.province_state
        day_df = pd.concat([day_df[colunns_of_interest], county_df[colunns_of_interest], whole_country_df])
    else:
        day_df = pd.concat([day_df[colunns_of_interest], whole_country_df])
    
#     day_df = day_df[colunns_of_interest]
    
    # increased granularity by neighborhood was added in Admin2 column 03-24-2020
    # but only want granularity down to province_region so collapse down and aggregate
    day_df = day_df.groupby(['country_region', 'province_state', 'date']).sum()
    day_df = day_df.reset_index()
    
    daily_dfs.append(day_df[colunns_of_interest])
    
daily_df = pd.concat(daily_dfs)
# make sure text columns are well cleaned and stripped of whitespace
daily_df.province_state = daily_df.province_state.str.strip()
daily_df.country_region = daily_df.country_region.str.strip()
daily_df.head(10)

,province_state,country_region,total_confirmed,total_deaths,total_recovered,date
0,,Afghanistan,1.0,0.0,0.0,2020-02-26
1,,Algeria,1.0,0.0,0.0,2020-02-26
2,From Diamond Princess,Australia,7.0,0.0,0.0,2020-02-26
3,New South Wales,Australia,4.0,0.0,4.0,2020-02-26
4,Queensland,Australia,5.0,0.0,1.0,2020-02-26
5,South Australia,Australia,2.0,0.0,2.0,2020-02-26
6,Victoria,Australia,4.0,0.0,4.0,2020-02-26
7,,Austria,2.0,0.0,0.0,2020-02-26
8,,Bahrain,33.0,0.0,0.0,2020-02-26
9,,Belgium,1.0,0.0,1.0,2020-02-26


In [17]:
# whole_country_df = daily_df.groupby(['country_region', 'date']).sum()
# whole_country_df = whole_country_df.reset_index()
# whole_country_df['province_state'] = [''] * whole_country_df.shape[0]
# whole_country_df.head(10)

In [18]:
# daily_df = pd.concat([daily_df, whole_country_df])
daily_df = daily_df.drop_duplicates()
locations = zip(daily_df.country_region.values, daily_df.province_state.values)
daily_df['location_id'] = [slugify_location(country_region, province_state)
                           for country_region, province_state in locations]

# sort by country_region, province_state, date
daily_df = daily_df.sort_values(['country_region', 'province_state', 'date'])
daily_df[(daily_df.country_region == 'US') & (daily_df.province_state == 'New York')]

,province_state,country_region,total_confirmed,total_deaths,total_recovered,date,location_id
183,New York,US,173.0,0.0,0.0,2020-03-10,us-new-york
190,New York,US,220.0,0.0,0.0,2020-03-11,us-new-york
192,New York,US,328.0,0.0,0.0,2020-03-12,us-new-york
205,New York,US,421.0,0.0,0.0,2020-03-13,us-new-york
219,New York,US,525.0,2.0,0.0,2020-03-14,us-new-york
227,New York,US,732.0,3.0,0.0,2020-03-15,us-new-york
240,New York,US,967.0,10.0,0.0,2020-03-16,us-new-york
244,New York,US,1706.0,13.0,0.0,2020-03-17,us-new-york
249,New York,US,2495.0,16.0,0.0,2020-03-18,us-new-york
256,New York,US,5365.0,34.0,0.0,2020-03-19,us-new-york


In [19]:
# get totals per location
max_date = daily_df.date.max()
rows_of_interest = daily_df.date == max_date
columns_of_interest = [
    'location_id',
    'total_confirmed',
    'total_deaths',
    'total_recovered'
]
location_totals_df = daily_df.loc[rows_of_interest, columns_of_interest]
location_totals_df = location_totals_df.groupby('location_id').sum()
location_totals_df['death_rate'] = location_totals_df.total_deaths / location_totals_df.total_confirmed * 100
location_totals_df['recovery_rate'] = location_totals_df.total_recovered / location_totals_df.total_confirmed * 100
location_totals_df.sort_values('total_confirmed', ascending=False).head(10)

,total_confirmed,total_deaths,total_recovered,death_rate,recovery_rate
location_id,,,,,
spain,126168.0,11947.0,34219.0,9.469121,27.121774
italy,124632.0,15362.0,20996.0,12.325887,16.846396
us-new-york,113833.0,3565.0,0.0,3.131781,0.000000
germany,96092.0,1444.0,26400.0,1.502727,27.473671
france,89953.0,7560.0,15438.0,8.404389,17.162296
china-hubei,67803.0,3207.0,63762.0,4.729879,94.040087
us-new-york-city-new-york,63306.0,1905.0,0.0,3.009193,0.000000
iran,55743.0,3452.0,19736.0,6.192706,35.405342
united-kingdom,41903.0,4313.0,135.0,10.292819,0.322173


In [20]:
# get totals per country / region
# country_totals_df = locations_df.join(location_totals_df)
# columns_of_interest = [
#     'country_region',
#     'total_confirmed',
#     'total_deaths',
#     'total_recovered',
# ]
# country_totals_df = country_totals_df.reset_index()
# country_totals_df = country_totals_df[columns_of_interest].groupby('country_region').sum()
# country_totals_df['death_rate'] = country_totals_df.total_deaths / country_totals_df.total_confirmed * 100
# country_totals_df['recovery_rate'] = country_totals_df.total_recovered / country_totals_df.total_confirmed * 100
# country_totals_df.sort_values('country_region').head(25)

In [21]:
# world_population_df = pd.read_csv('world_population.csv')
# world_population_df = world_population_df.set_index('country_region')
# world_population_df.head()

In [22]:
# # add population data to country totals
# country_totals_df = country_totals_df.join(world_population_df)
# country_totals_df.head(20)

In [20]:
daily_df[(daily_df.country_region == 'US') & (daily_df.location_id == 'us-new-york-city-new-york')]

,province_state,country_region,total_confirmed,total_deaths,total_recovered,date,location_id
2330,"New York City, New York",US,9654.0,63.0,0.0,2020-03-22,us-new-york-city-new-york
2324,"New York City, New York",US,12305.0,99.0,0.0,2020-03-23,us-new-york-city-new-york
2327,"New York City, New York",US,14904.0,131.0,0.0,2020-03-24,us-new-york-city-new-york
2331,"New York City, New York",US,17856.0,199.0,0.0,2020-03-25,us-new-york-city-new-york
2333,"New York City, New York",US,21873.0,281.0,0.0,2020-03-26,us-new-york-city-new-york
2334,"New York City, New York",US,25573.0,366.0,0.0,2020-03-27,us-new-york-city-new-york
2335,"New York City, New York",US,29776.0,517.0,0.0,2020-03-28,us-new-york-city-new-york
2335,"New York City, New York",US,33768.0,678.0,0.0,2020-03-29,us-new-york-city-new-york
2336,"New York City, New York",US,37453.0,790.0,0.0,2020-03-30,us-new-york-city-new-york
1668,"New York City, New York",US,43119.0,932.0,0.0,2020-03-31,us-new-york-city-new-york


In [21]:
# group by location and serialize each location dataset to a json file
# [ 
#   {
#     date: str,
#     province_state: str,
#     confirmed: int,
#     deaths: int,
#     recovered: int
#   }, ...
# ]

location_case_data = 'location_case_data'
if not os.path.exists(location_case_data):
    os.mkdir(location_case_data)

location_groups = daily_df.groupby(['location_id'])
for location_id, location_data in location_groups:
    location_data = location_data.sort_values('date')
    location_data.loc[:,'daily_confirmed'] = calc_differential(location_data.total_confirmed)
    location_data.loc[:,'daily_deaths'] = calc_differential(location_data.total_deaths)
    location_data.loc[:,'daily_recovered'] = calc_differential(location_data.total_recovered)
    
    location_days = []

    for idx, row in location_data.iterrows():
        data = row.to_dict()
        # dates don't serialize well in Python so, convert to strings
        data['date'] = data['date'].strftime('%Y-%m-%d')
        location_days.append(data)

    filename = f"{location_id}.json"
    file_path = os.path.join(location_case_data, filename)

    with open(file_path, 'w') as fo:
        json.dump(location_days, fo, indent=4)

    s3_url = upload_file_to_s3(s3_bucket, file_path, filename)


In [22]:
# locations_df.head(25)
# locations_df = locations_df.reset_index()
# locations_df.head()

In [23]:
locations_df = daily_df[['location_id', 'province_state', 'country_region']].drop_duplicates()
locations_df['cloud_resource'] = [cloud_resource_url(location_id, bucket_name) 
                                  for location_id in locations_df.location_id.values]
locations_df.head()

,location_id,province_state,country_region,cloud_resource
0,afghanistan,,Afghanistan,https://thecodinginterface-covid.s3.amazonaws....
1,albania,,Albania,https://thecodinginterface-covid.s3.amazonaws....
1,algeria,,Algeria,https://thecodinginterface-covid.s3.amazonaws....
2,andorra,,Andorra,https://thecodinginterface-covid.s3.amazonaws....
4,angola,,Angola,https://thecodinginterface-covid.s3.amazonaws....


In [24]:
locations_df[locations_df.province_state == 'Nebraska']

,location_id,province_state,country_region,cloud_resource
178,us-nebraska,Nebraska,US,https://thecodinginterface-covid.s3.amazonaws....


In [25]:
# create a list of dicts in the form:
# [ 
#   {
#     country_region: str,
#     province_state: str,
#     lat: float,
#     long: float,
#     filename: str,
#     cloud_resource: str
#   },
#    ...
# ]
locations = []
location_groups = locations_df.groupby(['location_id'])
for k, location_data in location_groups:
    for i, row in location_data.iterrows():
        data = row.to_dict()
        locations.append(data)
        if i < 5:
            print(data)

{'location_id': 'afghanistan', 'province_state': '', 'country_region': 'Afghanistan', 'cloud_resource': 'https://thecodinginterface-covid.s3.amazonaws.com/afghanistan.json'}
{'location_id': 'albania', 'province_state': '', 'country_region': 'Albania', 'cloud_resource': 'https://thecodinginterface-covid.s3.amazonaws.com/albania.json'}
{'location_id': 'algeria', 'province_state': '', 'country_region': 'Algeria', 'cloud_resource': 'https://thecodinginterface-covid.s3.amazonaws.com/algeria.json'}
{'location_id': 'andorra', 'province_state': '', 'country_region': 'Andorra', 'cloud_resource': 'https://thecodinginterface-covid.s3.amazonaws.com/andorra.json'}
{'location_id': 'angola', 'province_state': '', 'country_region': 'Angola', 'cloud_resource': 'https://thecodinginterface-covid.s3.amazonaws.com/angola.json'}
{'location_id': 'antigua-and-barbuda', 'province_state': '', 'country_region': 'Antigua and Barbuda', 'cloud_resource': 'https://thecodinginterface-covid.s3.amazonaws.com/antigua-an

In [26]:
# serialize locations to JSON file
with open('locations.json', 'w') as fo:
    json.dump(locations, fo, indent=4)

#!head -n 15 locations.json

## Country / Region Dashboard

Give user ability to select (aka drill down) into country, region, state, province

Show confirmed, deaths, recovered

Show time series of total confirmed

Show time series of total recovered

Show time series of total deaths

Show time series of daily new confirmed

Show time series of daily new recovered

Show time series of daily new deaths

Would be interesting to give a Gauge chart next to the daily graphs with an indicator of direction of n day movement (ie, over the last three days is new daily cases (deaths, confirmed, recovered) increasing, descreasing, maintaining)


## Location Comparisons

### Barcharts

Death Rates: select locations to include and date in time (includes checkbox to make percent of population)

Confirmed Counts: select locations to include and date in time (includes checkbox to make percent of population)


### Line Charts

Total Confirmed Cases: select locations to include and plot progression of cases since first case (includes checkbox to make percent of population)

New Daily Confirmed Cases: select locations to include and plot progression of new cases since first case in each location (includes checkbox to make percent of population)

Total Deaths: select locations to include and plot progression of deaths since first case (includes checkbox to make percent of population)

New Daily Deaths: select locations to include and plot progression of deaths since first  case in each location (includes checkbox to make percent of population)

In [23]:
locations_df.to_csv('locations.csv')